In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from opacus import PrivacyEngine
from model import SimpleNN
from utils import fgsm_attack


def train_model_with_dp_and_at(X_train, y_train, epsilon_target=3.0):
    X_tensor = torch.tensor(X_train).float()
    y_tensor = torch.tensor(y_train).long()
    train_loader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=64, shuffle=True)

    model = SimpleNN(X_tensor.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()

    privacy_engine = PrivacyEngine()
    model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        target_epsilon=epsilon_target,
        target_delta=1e-5,
        epochs=10,
        max_grad_norm=1.0,
    )

    model.train()
    for epoch in range(10):
        for batch_x, batch_y in train_loader:
            batch_x.requires_grad = True
            output = model(batch_x)
            loss = criterion(output, batch_y)
            model.zero_grad()
            loss.backward(retain_graph=True)
            adv_x = fgsm_attack(batch_x, 0.1, batch_x.grad)
            adv_output = model(adv_x)
            adv_loss = criterion(adv_output, batch_y)
            optimizer.zero_grad()
            adv_loss.backward()
            optimizer.step()

    final_epsilon = privacy_engine.get_epsilon(1e-5)
    return model, final_epsilon
